### **RAG를 이용한 설문지 생성**

In [28]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
import re
import win32com.client as win32
import os

In [29]:
# hwp2pdf
def hwp2pdf(folder_path, sav_path):
    
    # 한글 기본 설정 
    hwp = win32.gencache.EnsureDispatch("hwpframe.hwpobject")
    hwp.XHwpWindows.Item(0).Visible = False
    hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

    #파일 경로
    source_folder = os.path.abspath(folder_path)
    files = [f for f in os.listdir(source_folder) if re.match('.*[.]hwp', f)]
    
    print(f'hwp file {len(files)}개 변환 시작')
    
    for file in files:
        hwp.Open(os.path.join(source_folder, file))  # 파일 열기

        # 저장 경로
        filename = os.path.abspath(os.path.join(sav_path, file).replace('hwp', 'pdf'))
        print(filename)
        
        # PDF 저장 
        act = hwp.CreateAction("Print")
        pset = act.CreateSet()
        act.GetDefault(pset)
        pset.SetItem("FileName", filename)  
        pset.SetItem("PrinterName", "Microsoft Print to PDF") 
        pset.SetItem("PrintMethod", 0)   
        act.Execute(pset)
        
        hwp.Run("FileClose")  
        
        print(f'{file} 변환 완료')

    hwp.Quit() 

source_folder = './data/survey/hwp'
save_folder = './data/survey/pdf'

hwp2pdf(source_folder, save_folder)

hwp file 3개 변환 시작
c:\Users\bigdata\Dropbox\개인\Python\딥러닝\Model_implement\LangChain\0. Project\data\survey\pdf\22.pdf
22.hwp 변환 완료
c:\Users\bigdata\Dropbox\개인\Python\딥러닝\Model_implement\LangChain\0. Project\data\survey\pdf\33.pdf
33.hwp 변환 완료
c:\Users\bigdata\Dropbox\개인\Python\딥러닝\Model_implement\LangChain\0. Project\data\survey\pdf\[STI]_(설문지)_(사)21세기산업연구소_거창군 치유산업 종합계획 수립을 위한 설문조사(관광객 니즈 분석).pdf
[STI]_(설문지)_(사)21세기산업연구소_거창군 치유산업 종합계획 수립을 위한 설문조사(관광객 니즈 분석).hwp 변환 완료


In [30]:
# 문서 로드 
path = r"C:\Users\bigdata\Desktop\새 폴더\[STI]_(설문지)_(사)21세기산업연구소_거창군 치유산업 종합계획 수립을 위한 설문조사(관광객 니즈 분석).pdf"
loader = PyMuPDFLoader(path)

docs = loader.load()

In [31]:
# 텍스트 분리
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
split_docs = splitter.split_documents(docs)

In [32]:
# 임베딩 모델 생성
embedder = OpenAIEmbeddings(model = 'text-embedding-3-small')

# vector store 생성
vector_store = FAISS.from_documents(documents = split_docs, embedding=embedder)

# Retreiver 정의
retreiver = vector_store.as_retriever()

In [33]:
# 프롬프트를 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 

#주제:
{question} 

해당 주제에 관해 예시과 같은 포맷으로 설문지를 만들어줘 

#조건:
1. 설문지는 10문항으로 구성되어야함.
2. 만족도, 일반문항, 객관식문항, 객관식문항(복수응답),주관식문항이 포함되어야함.

예시) 
1. 귀하의 여행 스타일은 무엇입니까?

 국내여행   
② 해외여행

#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(model = 'gpt-4o-mini')

chain = (
    {'context' : retreiver, 'question' : RunnablePassthrough()}
    | prompt
    | llm
)

response = chain.stream('거창군 치유산업 종합계획 수립을 위한 설문지 -관광객 니즈 분석-')

for chunk in response:
    print(chunk.content, end = '', flush = True)

1. 귀하의 여행 스타일은 무엇입니까?
   - 국내여행  
   - 해외여행  

2. 거창군 여행 경험이 있습니까?
   - 있다  
   - 없다  

3. 거창군 관광지에 대해 얼마나 알고 있습니까?
   - 잘 알고 있다  
   - 알고 있다  
   - 조금 알고 있다  
   - 들어본 적 있다  
   - 들어본 적 없다  

4. 아래 항목에 대한 거창군의 인식을 평가해주십시오.  
   (1: 매우 좋음, 5: 전혀 좋지 않음)  
   - 자연경관  
   - 역사적 문화적 자원  
   - 관광지 이미지  
   - 독특성 및 고유성  
   - 숙박시설  
   - 관광지 접근성  
   - 레저시설  
   - 편의시설

5. 거창군 치유산업 발전을 위한 관심 있는 시설은 무엇입니까? (복수응답 가능)  
   - 체험 프로그램  
   - 테마파크  
   - 스파 및 온천  
   - 삼림욕 및 휴양림  

6. 귀하가 선호하는 시설 유형은 무엇입니까? (복수응답 가능)  
   - 실내체험시설  
   - 실외체험시설  
   - 실내체류형시설  
   - 실외체류형시설  

7. 거창군 방문 시 가장 중요하게 생각하는 요소는 무엇입니까?  
   - 가격  
   - 편안함  
   - 친환경성  
   - 시설 현대화  

8. 귀하의 연령대는 어떻게 되십니까?
   - 10대  
   - 20대  
   - 30대  
   - 40대  
   - 50대 이상  

9. 거창군의 관광 상품에 대한 만족도를 평가해주십시오.  
   - 매우 만족  
   - 만족  
   - 보통  
   - 불만족  
   - 매우 불만족  

10. 마지막으로 거창군 치유산업 발전을 위한 귀하의 의견을 자유롭게 작성하여 주십시오.  
   - [주관식] 